In [1]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [2]:
import torch
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

# Import model
from architectures.pgnniv_pod import PGNNIVPOD

In [3]:
# Dataset
dataset = 'non_linear'
N_data = 1000
noise = 0

data_name = dataset + '_' + str(N_data) + '_' + str(noise)

In [4]:
# Model
model = 'POD'
n_modes = 10

model_name = model + '_model_' + str(n_modes)

In [5]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name)

In [6]:
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_1000_0
Folder successfully created at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear_1000_0/POD_model_10


In [7]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_1000_0/non_linear_1000_0.pkl


In [8]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [9]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [10]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [11]:
num_modes = 20

U_train, S_train, Vt_train = torch.linalg.svd(y_train.values.detach().squeeze().to('cpu').view(y_train.values.detach().shape[0], -1), full_matrices=False)

U_reduced_train = U_train[:, :num_modes]
S_reduced_train = S_train[:num_modes]
Vt_reduced_train = Vt_train[:num_modes, :]

POD_base = Vt_reduced_train.to(DEVICE)

In [12]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters 
n_filters_explanatory = 5

In [13]:
# Load model and the optimizer
model = PGNNIVPOD(input_shape, predictive_layers, POD_base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 10000

batch_size = 64
n_checkpoints = 5

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 7.620e+08, Test loss: 9.960e+08, MSE(e): 7.476e+01, MSE(pi1): 1.198e+03, MSE(pi2): 2.841e+01, MSE(pi3): 2.415e+01
Epoch 100, Train loss: 1.989e+07, Test loss: 3.581e+07, MSE(e): 1.982e+00, MSE(pi1): 4.432e+00, MSE(pi2): 9.572e-01, MSE(pi3): 3.275e-01
Epoch 200, Train loss: 1.529e+06, Test loss: 2.542e+06, MSE(e): 1.493e-01, MSE(pi1): 1.813e+00, MSE(pi2): 8.966e-02, MSE(pi3): 1.695e-01
Epoch 300, Train loss: 2.127e+05, Test loss: 5.032e+05, MSE(e): 1.817e-02, MSE(pi1): 1.411e+00, MSE(pi2): 1.044e-02, MSE(pi3): 1.689e-01
Epoch 400, Train loss: 9.998e+04, Test loss: 2.365e+05, MSE(e): 7.072e-03, MSE(pi1): 1.271e+00, MSE(pi2): 4.426e-03, MSE(pi3): 1.655e-01
Epoch 500, Train loss: 7.234e+04, Test loss: 1.326e+05, MSE(e): 4.646e-03, MSE(pi1): 1.120e+00, MSE(pi2): 3.096e-03, MSE(pi3): 1.468e-01
Epoch 600, Train loss: 4.049e+04, Test loss: 7.784e+04, MSE(e): 2.931e-03, MSE(pi1): 5.339e-01, MSE(pi2): 1.995e-03, MSE(pi3): 5.841e-02
Epoch 700, 

In [14]:
# Parametros de entrenamiento
start_epoch = 800
n_epochs = 1000

batch_size = 64 
n_checkpoints = 10

second_lr = 3e-4

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 800.



KeyboardInterrupt

